In [26]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

# Database connections 

In [27]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# SQL Query

In [28]:
# Consulta de ejemplo para validar los datos cargados
query = """
SELECT 
    ha.servicio_id,
    ha.key_dim_fecha,
    ha.key_dim_cliente,
    ha.key_dim_mensajero,
    ha.key_dim_hora,
    ds.key_dim_sede,
    EXTRACT(HOUR FROM ha.hora_iniciado::time) as hora_servicio
FROM hecho_entrega_acumulado ha
JOIN dim_cliente dc ON ha.key_dim_cliente = dc.key_dim_cliente
JOIN dim_sede ds ON dc.cliente_id = ds.sede_id
WHERE ha.hora_iniciado IS NOT NULL
"""

# Extract

In [29]:
# Leer datos
df_servicios = pd.read_sql(query, bodega_conn)

In [30]:
print("Total servicios extraídos:", len(df_servicios))
print("\nDistribución de horas:")
print(df_servicios['hora_servicio'].value_counts().sort_index())

Total servicios extraídos: 28428

Distribución de horas:
hora_servicio
0.0      146
1.0      164
2.0      156
3.0      107
4.0      103
5.0      138
6.0      901
7.0     1066
8.0     2636
9.0     3391
10.0    2932
11.0    3297
12.0    1719
13.0    1465
14.0    2573
15.0    2658
16.0    2122
17.0    1357
18.0     442
19.0     346
20.0     224
21.0     176
22.0     154
23.0     155
Name: count, dtype: int64


In [31]:
hecho_hora = df_servicios.groupby([
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_sede',
    'key_dim_mensajero',
    'key_dim_hora',
    'servicio_id'
]).size().reset_index(name='cantidad_servicios')

# Verifications

In [36]:
print("\nEstadísticas de agregación:")
print(f"Total registros agregados: {len(hecho_hora)}")
print("\nDistribución de cantidad de servicios:")
print(hecho_hora['cantidad_servicios'].value_counts())


Muestra de datos a cargar:
   key_dim_fecha  key_dim_cliente  key_dim_sede  key_dim_mensajero  \
0          261.0                7            16                0.0   
1          261.0                7            16               13.0   
2          261.0                7            16               13.0   
3          261.0                7            16               13.0   
4          261.0                7            16               13.0   

   key_dim_hora  servicio_id  cantidad_servicios       saved  
0          16.0            7                   1  2024-11-10  
1          16.0            8                   1  2024-11-10  
2          16.0            9                   1  2024-11-10  
3          16.0           10                   1  2024-11-10  
4          16.0           11                   1  2024-11-10  

Información de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27701 entries, 0 to 27700
Data columns (total 8 columns):
 #   Column              Non-Null Count

In [37]:
# Agregar fecha de carga
hecho_hora['saved'] = date.today()

701

In [36]:
# Verificar datos antes de cargar
print("\nMuestra de datos a cargar:")
print(hecho_hora.head())
print("\nInformación de columnas:")
print(hecho_hora.info())


Muestra de datos a cargar:
   key_dim_fecha  key_dim_cliente  key_dim_sede  key_dim_mensajero  \
0          261.0                7            16                0.0   
1          261.0                7            16               13.0   
2          261.0                7            16               13.0   
3          261.0                7            16               13.0   
4          261.0                7            16               13.0   

   key_dim_hora  servicio_id  cantidad_servicios       saved  
0          16.0            7                   1  2024-11-10  
1          16.0            8                   1  2024-11-10  
2          16.0            9                   1  2024-11-10  
3          16.0           10                   1  2024-11-10  
4          16.0           11                   1  2024-11-10  

Información de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27701 entries, 0 to 27700
Data columns (total 8 columns):
 #   Column              Non-Null Count

# Load

In [40]:
# Guardar en la bodega
hecho_hora.to_sql('hecho_entrega_servicio_hora', bodega_conn, if_exists='replace', index_label='key_hecho_entrega_servicio_hora')

701

# SQL Querys to response requirements

In [ ]:
# Pregunta 3: ¿A qué hora los mensajeros están más ocupados?
"""
WITH servicios_por_hora AS (
    SELECT 
        dh.hora,
        dh.periodo_dia,
        COUNT(h.servicio_id) as total_servicios,
        ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER ()) * 100, 2) as porcentaje
    FROM hecho_entrega_servicio_hora h
    JOIN dim_hora dh ON h.key_dim_hora = dh.key_dim_hora
    GROUP BY dh.hora, dh.periodo_dia
)
SELECT 
    hora,
    periodo_dia,
    total_servicios,
    porcentaje || '%' as porcentaje_servicios
FROM servicios_por_hora
ORDER BY total_servicios DESC;
"""

"\nWITH servicios_por_hora AS (\n    SELECT \n        dh.hora,\n        dh.periodo_dia,\n        COUNT(h.servicio_id) as total_servicios,\n        ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER ()) * 100, 2) as porcentaje\n    FROM hecho_entrega_servicio_hora h\n    JOIN dim_hora dh ON h.key_dim_hora = dh.key_dim_hora\n    GROUP BY dh.hora, dh.periodo_dia\n)\nSELECT \n    hora,\n    periodo_dia,\n    total_servicios,\n    porcentaje || '%' as porcentaje_servicios\nFROM servicios_por_hora\nORDER BY total_servicios DESC;\n"

In [ ]:
# Pregunta 4: ¿Número de servicios solicitados por cliente y por mes?
"""
SELECT 
    c.cliente_id,
    c.nombre as nombre_cliente,
    EXTRACT(YEAR FROM df.fecha)::integer as año,
    CASE EXTRACT(MONTH FROM df.fecha)::integer
        WHEN 1 THEN 'Enero'
        WHEN 2 THEN 'Febrero'
        WHEN 3 THEN 'Marzo'
        WHEN 4 THEN 'Abril'
        WHEN 5 THEN 'Mayo'
        WHEN 6 THEN 'Junio'
        WHEN 7 THEN 'Julio'
        WHEN 8 THEN 'Agosto'
        WHEN 9 THEN 'Septiembre'
        WHEN 10 THEN 'Octubre'
        WHEN 11 THEN 'Noviembre'
        WHEN 12 THEN 'Diciembre'
    END as mes,
    COUNT(h.servicio_id) as total_servicios
FROM hecho_entrega_servicio_hora h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente
GROUP BY 
    c.cliente_id, 
    c.nombre,
    EXTRACT(YEAR FROM df.fecha),
    EXTRACT(MONTH FROM df.fecha)
ORDER BY 
    c.cliente_id, 
    año, 
    EXTRACT(MONTH FROM df.fecha);
"""

"\nSELECT \n    c.cliente_id,\n    c.nombre as nombre_cliente,\n    EXTRACT(YEAR FROM df.fecha)::integer as año,\n    CASE EXTRACT(MONTH FROM df.fecha)::integer\n        WHEN 1 THEN 'Enero'\n        WHEN 2 THEN 'Febrero'\n        WHEN 3 THEN 'Marzo'\n        WHEN 4 THEN 'Abril'\n        WHEN 5 THEN 'Mayo'\n        WHEN 6 THEN 'Junio'\n        WHEN 7 THEN 'Julio'\n        WHEN 8 THEN 'Agosto'\n        WHEN 9 THEN 'Septiembre'\n        WHEN 10 THEN 'Octubre'\n        WHEN 11 THEN 'Noviembre'\n        WHEN 12 THEN 'Diciembre'\n    END as mes,\n    COUNT(h.servicio_id) as total_servicios\nFROM hecho_entrega_servicio_hora h\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nJOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente\nGROUP BY \n    c.cliente_id, \n    c.nombre,\n    EXTRACT(YEAR FROM df.fecha),\n    EXTRACT(MONTH FROM df.fecha)\nORDER BY \n    c.cliente_id, \n    año, \n    EXTRACT(MONTH FROM df.fecha);\n"

In [ ]:
# Pregunta 6: ¿Cuáles son las sedes que más servicios solicitan por cada cliente?
"""
WITH totales_cliente AS (
    SELECT 
        c.cliente_id,
        c.nombre as nombre_cliente,
        COUNT(h.servicio_id) as total_servicios_cliente
    FROM hecho_entrega_servicio_hora h
    JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente
    GROUP BY c.cliente_id, c.nombre
),
servicios_sede AS (
    SELECT 
        c.cliente_id,
        c.nombre as nombre_cliente,
        s.sede_id,
        s.nombre_sede,
        COUNT(h.servicio_id) as total_servicios_sede,
        RANK() OVER (PARTITION BY c.cliente_id ORDER BY COUNT(h.servicio_id) DESC) as ranking
    FROM hecho_entrega_servicio_hora h
    JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente
    JOIN dim_sede s ON h.key_dim_sede = s.key_dim_sede
    GROUP BY 
        c.cliente_id,
        c.nombre,
        s.sede_id,
        s.nombre_sede
)
SELECT 
    ss.cliente_id,
    ss.nombre_cliente,
    ss.sede_id,
    ss.nombre_sede,
    ss.total_servicios_sede as servicios_en_sede,
    tc.total_servicios_cliente as total_servicios_cliente,
    ROUND((ss.total_servicios_sede::numeric / tc.total_servicios_cliente::numeric) * 100, 2) || '%' as porcentaje_del_total
FROM servicios_sede ss
JOIN totales_cliente tc ON ss.cliente_id = tc.cliente_id
WHERE ss.ranking = 1
ORDER BY ss.total_servicios_sede DESC;
"""

"\nWITH totales_cliente AS (\n    -- Primero calculamos el total de servicios por cliente\n    SELECT \n        c.cliente_id,\n        c.nombre as nombre_cliente,\n        COUNT(h.servicio_id) as total_servicios_cliente\n    FROM hecho_entrega_servicio_hora h\n    JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente\n    GROUP BY c.cliente_id, c.nombre\n),\nservicios_sede AS (\n    SELECT \n        c.cliente_id,\n        c.nombre as nombre_cliente,\n        s.sede_id,\n        s.nombre_sede,\n        COUNT(h.servicio_id) as total_servicios_sede,\n        RANK() OVER (PARTITION BY c.cliente_id ORDER BY COUNT(h.servicio_id) DESC) as ranking\n    FROM hecho_entrega_servicio_hora h\n    JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente\n    JOIN dim_sede s ON h.key_dim_sede = s.key_dim_sede\n    GROUP BY \n        c.cliente_id,\n        c.nombre,\n        s.sede_id,\n        s.nombre_sede\n)\nSELECT \n    ss.cliente_id,\n    ss.nombre_cliente,\n    ss.sede_id,\n    ss.nomb